In [1]:
from urllib.request import urlopen, urlretrieve, Request
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup

import pandas as pd
import selenium

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
#  Dados de desempenho do time masculino de futebol do Cruzeiro 2014 - 2022
navegador = webdriver.Chrome()

In [3]:
a = 0  
url = "https://fbref.com/pt/equipes/03ff5eeb/Cruzeiro-Estatisticas"

In [4]:
while a in range(0,9):
    
    navegador.get(url)
    navegador.find_element("xpath", "/html/body/div[2]/div[3]/div[1]/div[2]/div/a/div").click()
    
    url = navegador.current_url
    headers ={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36"}

    req = Request(url, headers=headers)
    response = urlopen(req)
    html = response.read()
    soup = BeautifulSoup(html,'html.parser')
    lista = soup.find('table').findNext('table')

#  Ações e suas respectivas quantidades
    registro = lista.findAll('th', class_ = 'left') #  os registros da tabela
    qtd_1 = range(int(len(registro)))
    resumo = []

#  Informações da primeira linha que não entram no loop
    data =  lista.find('th', class_='left').getText()
    horario = lista.find('td').contents[0]

#  Para cada linha (i) em registro, obtenha os seguintes conteúdos:
    for i in registro:
        cruzeiro ={}
        camp = horario.findNext('td').contents[0]
        rodada = camp.findNext('td').contents[0]
        dia = rodada.findNext('td').contents[0]
        local = dia.findNext('td').contents[0]
        resultado = local.findNext('td').contents[0]
        gp = resultado.findNext('td').contents[0]
        gc = gp.findNext('td').contents[0]
        oponente = gc.findNext('td').contents[0]

        
#  Armazene as informações em cruzeiro da seguinte forma:
        cruzeiro['Data'] = data
        cruzeiro['Horário'] = horario
        cruzeiro['Campeonato'] = camp
        cruzeiro['Rodada'] = rodada
        cruzeiro['Dia'] = dia
        cruzeiro['Local']= local
        cruzeiro['Resultado'] = resultado
        cruzeiro['GP'] = gp
        cruzeiro['GC'] = gc
        cruzeiro['Oponente'] = oponente

        
#  Armazene cruzeiro na lista vazia criada anteriormente
        resumo.append(cruzeiro)
    
#  A última linha deve dar erro, pois não tem próxima etapa. Por isso, é necessária uma exceção:
        try:
            data = i
            horario = data.findNext('td').contents[0]
        except:
            pass
    
    a += 1

In [5]:
#  Cria data frame com dados extraídos anteriormente
df = pd.DataFrame(resumo)
 
display(df)

,Data,Horário,Campeonato,Rodada,Dia,Local,Resultado,GP,GC,Oponente
0,2014-02-12,[19:00],[Libertadores],[Segunda fase],qua,Visitante,D,1,2,[[pe]]
1,[[2014-02-12]],[19:00],[Libertadores],[Segunda fase],qua,Visitante,D,1,2,[[pe]]
2,[[2014-02-25]],[17:30],[Libertadores],[Segunda fase],ter,Em casa,V,5,1,[[cl]]
3,[[2014-03-11]],[19:00],[Libertadores],[Segunda fase],ter,Visitante,D,0,2,[[uy]]
4,[[2014-03-20]],[22:00],[Libertadores],[Segunda fase],qui,Em casa,E,2,2,[[uy]]
5,[[2014-04-03]],[20:00],[Libertadores],[Segunda fase],qui,Visitante,V,2,0,[[cl]]
6,[[2014-04-09]],[21:00],[Libertadores],[Segunda fase],qua,Em casa,V,3,0,[[pe]]
7,[[2014-04-16]],[22:00],[Libertadores],[Oitavas de final],qua,Em casa,E,1,1,[[py]]
8,[[2014-04-20]],[16:00],[Série A],[Rodada da semana 1],dom,Visitante,V,2,1,[Bahia]
9,[[2014-04-27]],[16:00],[Série A],[Rodada da semana 2],dom,Em casa,E,1,1,[São Paulo]
